# **Spin** calculation

IDEA:
- take in consideration only the ball for each frame
- draw a lot of points on the ball
- take the motion of these points between consecutive frames (optical flow)
- from this motion we extract the direction of the rotation and so the axis of rotation

In [1]:
import cv2
import numpy as np
import pandas as pd
from pathlib import Path

VIDEO_NUMBER = "3"
PROJECT_ROOT = Path().resolve().parent.parent
INPUT_VIDEO_PATH = str(PROJECT_ROOT / "data" / f"recording_{VIDEO_NUMBER}" / f"Recording_{VIDEO_NUMBER}.mp4")
INPUT_CSV_PATH = str(PROJECT_ROOT / "notebook" / "ball_detection" / "intermediate_data" / f"Circle_positions_cleaned_{VIDEO_NUMBER}.csv")

# --- open video & CSV ---
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)
if not cap.isOpened():
    raise IOError("Could not open video.")
df = pd.read_csv(INPUT_CSV_PATH)
if df.empty:
    raise ValueError("CSV is empty.")

# --- prepare writer ---
fps = cap.get(cv2.CAP_PROP_FPS)
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out_path = str(PROJECT_ROOT / "data" / f"recording_{VIDEO_NUMBER}" / "test_video" / f"Spin_test_{VIDEO_NUMBER}.mp4")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(str(out_path), fourcc, fps, (W, H))

# Prepare list to log axis endpoints
log = []

# Read frame0 and frame1
ret_curr, curr = cap.read()
ret_next, next_ = cap.read()
frame_idx = 0

while ret_curr and ret_next:
    vis = curr.copy()
    pA, pB = (np.nan, np.nan), (np.nan, np.nan)

    # Only compute if we have valid circle data
    if frame_idx < len(df) and not df.iloc[frame_idx][['x','y','radius']].isna().any():
        x, y, r = df.iloc[frame_idx][['x','y','radius']].astype(int)
        center = np.array([x, y])
        off = 2
        x0, x1 = max(x-r-off,0), min(x+r+off, W)
        y0, y1 = max(y-r-off,0), min(y+r+off, H)

        roi1 = curr[y0:y1, x0:x1]
        roi2 = next_[y0:y1, x0:x1]
        g1 = cv2.cvtColor(roi1, cv2.COLOR_BGR2GRAY)
        g2 = cv2.cvtColor(roi2, cv2.COLOR_BGR2GRAY)
        c_roi = np.array([x - x0, y - y0])

        axes = []
        for _ in range(3):
            pts = []
            while len(pts) < 1000:
                xi = np.random.randint(0, g1.shape[1])
                yi = np.random.randint(0, g1.shape[0])
                if (xi-c_roi[0])**2 + (yi-c_roi[1])**2 <= (r-2)**2:
                    pts.append([xi, yi])
            p0 = np.array(pts, dtype=np.float32).reshape(-1,1,2)
            if p0.size == 0:
                continue

            p1, st, _ = cv2.calcOpticalFlowPyrLK(
                g1, g2, p0, None,
                winSize=(15,15), maxLevel=2,
                criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT, 10, 0.03)
            )

            old3d, new3d = [], []
            for o, n, s in zip(p0.reshape(-1,2), p1.reshape(-1,2), st.reshape(-1)):
                if s:
                    ox, oy = o - c_roi
                    nx, ny = n - c_roi
                    oz = np.sqrt(max(r*r - ox*ox - oy*oy, 0))
                    nz = np.sqrt(max(r*r - nx*nx - ny*ny, 0))
                    old3d.append([ox, oy, oz])
                    new3d.append([nx, ny, nz])

            if len(old3d) < 3:
                continue

            old3d = np.array(old3d)
            new3d = np.array(new3d)
            Hmat = old3d.T @ new3d
            U, S, Vt = np.linalg.svd(Hmat)
            R = Vt.T @ U.T
            if np.linalg.det(R) < 0:
                Vt[-1,:] *= -1
                R = Vt.T @ U.T

            theta = np.arccos(np.clip((np.trace(R)-1)/2, -1,1))
            if np.sin(theta) != 0:
                ax = np.array([R[2,1]-R[1,2],
                               R[0,2]-R[2,0],
                               R[1,0]-R[0,1]])
                ax /= (2*np.sin(theta))
                ax /= np.linalg.norm(ax)
                axes.append(ax)

        if axes:
            avg = np.mean(np.vstack(axes), axis=0)
            avg /= np.linalg.norm(avg)
            if avg[2] != 0:
                a2 = avg[:2]/np.linalg.norm(avg[:2])
                pA = tuple((center + a2 * r).astype(int))
                pB = tuple((center - a2 * r).astype(int))
                cv2.line(vis, pA, pB, (255,255,0), 2)

    # log the endpoints for this frame
    log.append({
        'frame': frame_idx,
        'pA_x': pA[0], 'pA_y': pA[1],
        'pB_x': pB[0], 'pB_y': pB[1]
    })

    out.write(vis)

    # advance frames
    frame_idx += 1
    curr = next_
    ret_next, next_ = cap.read()
    ret_curr = curr is not None

# write last frame if exists
if ret_curr:
    vis = curr.copy()
    log.append({'frame': frame_idx,
                'pA_x': np.nan, 'pA_y': np.nan,
                'pB_x': np.nan, 'pB_y': np.nan})
    out.write(vis)

cap.release()
out.release()

# Save the log to CSV
log_df = pd.DataFrame(log)
csv_path = str(PROJECT_ROOT / "notebook" / "spin" / "intermediate_data" / f"Spin_data_{VIDEO_NUMBER}.csv")
log_df.to_csv(csv_path, index=False)

print(f"Video saved to {out_path} \nAxis log saved to {csv_path}")

Video saved to C:\Users\miche\OneDrive\Documenti\GitHub\bowling-analysis\data\recording_3\test_video\Spin_test_3.mp4 
Axis log saved to C:\Users\miche\OneDrive\Documenti\GitHub\bowling-analysis\notebook\spin\intermediate_data\Spin_data_3.csv
